# REM API Demo With `requests`

This notebook demonstrates how we can call the REM API to get an estimate
for the savings we can expect if we make an electrical upgrade to a home
at a particular address.

## Imports and Configuration

In [1]:
import requests
from pathlib import Path

In [2]:
HOST = "https://api.rewiringamerica.org"
REM_ADDRESS_URL = f"{HOST}/api/v1/rem/address"

API_KEY = None  # Put your API key here, or better yet in the file ~/.rwapi/api_key.txt

In [3]:
if API_KEY is None:
    api_key_path = Path.home() / ".rwapi" / "api_key.txt"

    if api_key_path.is_file():
        with open(api_key_path) as f:
            API_KEY = f.read().strip()

## Parameters

Address we are interested in and the upgrade we want to do.

In [4]:
address = '8009 Belmont Ave., Lubbock, TX 79424'
upgrade = "high_eff_hp_elec_backup"
heating_fuel = 'natural_gas'

## Make the Request

In [5]:
headers = {"Authorization": f"Bearer {API_KEY}"}

response = requests.get(
    url=REM_ADDRESS_URL,
    headers=headers,
    params=dict(
        address=address, upgrade=upgrade, heating_fuel=heating_fuel
    )
)

## Pull out the results

We are specifically interested in the total dollar savings.

In [6]:
data = response.json()

In [7]:
annual_savings = -data['fuel_results']['total']['delta']['cost']['mean']['value']

f"Expected annual savings: ${annual_savings:.2f}"

'Expected annual savings: $630.39'

## Modeling Only Baseline in REM

REM typically provides three estimates for a home: one without an upgrade, one with the requested upgrade, and the difference (delta) between them.
For a detailed explanation of these statistics, refer to the [All About REM Statistics notebook](https://github.com/rewiringamerica/api_demos/blob/main/notebooks/All%20About%20REM%20Statistics.ipynb).

A special case occurs when `baseline` is requested as the upgrade. In this case, REM returns a subset of the expected data structure.
For each `fuel_type` within `fuel_results`, the `upgrade` and `delta` fields will not be populated. 
Requesting `baseline` tells REM to perform estimation without applying any upgrades.

Performing a request with only `baseline` can be done similarly to other requests.

In [ ]:
headers = {"Authorization": f"Bearer {API_KEY}"}

response = requests.get(
    url=REM_ADDRESS_URL,
    headers=headers,
    params=dict(
        address=address, upgrade=upgrade, heating_fuel=heating_fuel
    )
)

### Pull out the results for a baseline request

We will extract the `fuel_results`, as we have done previously. In the response, `baseline` contains valid values, while `upgrade` and `delta` are null, as expected since no upgrade was applied to the home.

In [ ]:
data = response.json()
fuel_results = data["fuel_results"]

In [ ]:
fuel_results['fuel_oil']['baseline']

In [ ]:
fuel_results['fuel_oil']['upgrade']

In [ ]:
fuel_results['fuel_oil']['delta']